In [1]:
# KNN + Boarder Detection + Normal Direction Constrains

# 1) KNN search: several candidates of corresponding points
# 2) Boarder Detection + Normal Direction constrains => remove some bad correspondences

In [2]:
import math
import pickle
import os
import time
import scipy
import numpy as np
from scipy.optimize import minimize
from sklearn.neighbors import NearestNeighbors

## SMPL model
data_root   = '/home/yan/Data2/NOMO_Project_Summary/'
root_folder = data_root + 'MVA/MVA_rebuttal_code/Shape_Parameter_Optimization/'
import sys
sys.path.append(root_folder+'SMPL/SMPL_python_v.1.0.0/smpl/')
from smpl_webuser.serialization import load_model

smpl_model  = root_folder + 'SMPL/SMPL_python_v.1.0.0/smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'
out_folder  = root_folder + 'optimized_mesh/dataset01/'
scan_folder =  data_root + 'Data/TC2_scan/ori_scan/re_name/dataset01/male/'

# pre-process initial params
# param_folder = '/home/yan/Data2/3D_Body_Reconstruction/Dataset/scans/Optimized_Registered_NOMO3D_Dataset1/Original_A_Posed/parameter_male/'
param_folder = None
# load SMPL model
model = load_model(smpl_model)
scan_list = os.listdir(scan_folder)

deg2rad = 1.0 / 180 * math.pi
bnds = ((-20*deg2rad, 10*deg2rad), (-15*deg2rad,  15*deg2rad), (-90*deg2rad, -30*deg2rad), 
        (-20*deg2rad, 10*deg2rad), (-15*deg2rad,  15*deg2rad), (30*deg2rad,   90*deg2rad),
        (0*deg2rad,   30*deg2rad), (-30*deg2rad, 0*deg2rad), (0*deg2rad,    15*deg2rad),  
        (0*deg2rad,   30*deg2rad), (0*deg2rad,  30*deg2rad), (-15*deg2rad,   10*deg2rad),  
        (-10*deg2rad,  10*deg2rad), (-10*deg2rad,  10*deg2rad), (-5*deg2rad,    15*deg2rad), 
        (-10*deg2rad,   10*deg2rad), (-10*deg2rad,  10*deg2rad), (-15*deg2rad,   5*deg2rad),
        (-5*deg2rad,   15*deg2rad), (-5*deg2rad,  15*deg2rad), (-10*deg2rad,    10*deg2rad),
        (-15*deg2rad,  5*deg2rad), (-15*deg2rad, 5*deg2rad), (-15*deg2rad,   5*deg2rad), 
        (-15*deg2rad,  5*deg2rad), (-5*deg2rad,  15*deg2rad), (-5*deg2rad,    15*deg2rad),
        (-3, 3), (-3, 3), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5), (-5, 5))

In [3]:
def loadTarget(scanFile):
    ''' load NOMO3D scans, return Vertices and Faces'''
    with open(scanFile, 'r') as fp:
        content = fp.readlines()
    content = [x.strip() for x in content]

    targetV, targetF = [], []
    for line in content:
        CC = line.split()
        if len(CC) > 0:
            if CC[0] == 'v':
                targetV.append([float(CC[1]), float(CC[2]), float(CC[3])])
            elif CC[0] == 'f':
                targetF.append([int(CC[1]), int(CC[2]), int(CC[3])])

    targetV = np.array(targetV)
    targetV = targetV - np.mean(targetV, 0)
    targetF = np.array(targetF, dtype=int) - 1 # .obj file, is 0-based, started from 0
    
    return targetV, targetF

def loadInitParam(paramFile):
    '''load initial parameters, including shape and pose from the previous experiments'''
    if paramFile == None:
        init_betas = np.zeros(10)
        init_pose = np.zeros(72)
        deg2rad = 1.0 / 180 * math.pi
        init_pose[48:60] = np.multiply(deg2rad, [-12, 2, -69, -12,-2, 69, 16, -18, 4, 16, 18, -4])
        init_pose[3:18] = np.multiply(deg2rad, [1, 0, 9, 1, 0, -8, 4, 4, 0, -3, -2, -2, -3, 2, 2])
    else:
        with open(paramFile, 'rb') as f:
            param_data = pickle.load(f)
        init_pose = param_data[:72]
        init_betas = param_data[72:]
    
    return init_pose, init_betas

def normalize_v3(arr):
    '''Normalize a numpy array of 3 component vectors shape=(n, 3)'''
    lens = np.sqrt(arr[:, 0]**2 + arr[:, 1]**2 + arr[:, 2]**2)
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def getNormals(V, F):
    ''' Creating a normal per triangle, is done by the cross product of two vectors on this triangle.
    Creating smooth normals, i.e. one normal per vertex is done 
    by averaging the normal of all the triangles one vertex is part of.
    
    https://sites.google.com/site/dlampetest/python/calculating-normals-of-a-triangle-mesh-using-numpy'''
    
    norm = np.zeros(V.shape, dtype=V.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = V[F] 
    # Calculate the normal for all the triangles, 
    # by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle 
    n = np.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
    n = normalize_v3(n)
    # Multiple triangles would then contribute to every vertex
    norm[F[:, 0]] += n
    norm[F[:, 1]] += n
    norm[F[:, 2]] += n
    norm = normalize_v3(norm)
    
    return norm

def getBoarderV(Faces):
    '''the boarder of the mesh are those edges which belong to only one polygon'''
    boundV = []
    edges01 = np.array([Faces[:, 0], Faces[:, 1]], dtype=int).T
    edges02 = np.array([Faces[:, 0], Faces[:, 2]], dtype=int).T
    edges03 = np.array([Faces[:, 1], Faces[:, 2]], dtype=int).T
    edge = np.sort(np.concatenate((edges01, edges02, edges03), axis=0))
    
    unique_e, n_counts = np.unique(edge, axis=0, return_counts=True)
    boarder_edge = unique_e[n_counts==1, :]
    boundV = np.unique(boarder_edge.flatten())
    
    return boundV

def getDist(dist_arr, idx_arr, K, modelN):
    ''' get the weighted mean distance from the model to scan
            dist_arr : 6890*K
            idx_arr  : 6890*K
                K    : the number of KNN neighbour, K > 1
            modelN   : 6890*3, the normals of modelV
    '''
    angle_arr = np.zeros((6890, K), dtype=float)
    for kk in range(K):
        correspN = targetNorm[idx_arr[:, kk], :]
        norm_dot = np.einsum('ij,ij->i', modelN, correspN) # dot
        angle_arr[:, kk] = np.absolute(np.arccos(np.clip(norm_dot, -1.0, 1.0))) # 6890*1
    # find the smallest angle in each row
    col_idx = np.argmin(angle_arr, axis=1)          # (6890,) = [0 1 1 0 0 0 1 1 ...]
    idx     = np.choose(col_idx, idx_arr.T)
    angle   = np.choose(col_idx, angle_arr.T)
    dist    = np.choose(col_idx, dist_arr.T)
    
    W = np.ones((6890,), dtype=float)                # 0 for bad correspondences, otherwise 1  
    W[np.nonzero(dist > 0.1)]             = 0        # 10 cm
    W[np.nonzero(angle > math.pi / 4)]    = 0        # 45 degree
    W[np.nonzero(np.isin(idx, boarderV))] = 0        # boarder vertices

    return np.sum(np.multiply(dist, W)) / np.count_nonzero(W)

def meshDist(param):
    '''the distance between the SMPL model and the target scan'''
    
    model.betas[:]    = param[27:]
    model.pose[48:60] = param[:12]
    model.pose[3:18]  = param[12:27]
    
    modelF = model.f # 0-based, started from 0
    modelV = model.r - np.mean(model.r, 0)
    modelNorm = getNormals(modelV, modelF)
    
    N_neigh = 5                                       # KNN correspondence search,  find the cloest corresponding
    t_dist, t_idx = kdtree.query(modelV, k=N_neigh)   # t_dist: 6890*K, t_idx: 6890*K
    
    return getDist(t_dist, t_idx, N_neigh, modelNorm) * 1000 # [mm]

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

for scan in scan_list:
    scan_name = scan[:-4]

    targetV, targetF = loadTarget(scan_folder+scan_name+'.obj') # registered mesh, (6890, 3)
    boarderV   = getBoarderV(targetF)                             # the vertices on the boarder edges
    targetNorm = getNormals(targetV, targetF)                   # the normal for every target vertices
    
    kdtree = scipy.spatial.cKDTree(targetV, leafsize=6890)      # KNN search
    
#     paramFile = param_folder + scan_name + '.pkl'               # None for female samples                       
    paramFile = None
    init_pose, init_betas = loadInitParam(paramFile)            # female does not have the pre-param
    
    init_param        = np.zeros(37)                            # 27 part pose parameters + 10 shape param
    init_param[:12]   = init_pose[48:60]
    init_param[12:27] = init_pose[3:18]
    init_param[27:]   = init_betas
    
    start_time = time.time()
    res = minimize(meshDist, init_param, method='L-BFGS-B', bounds=bnds, 
                   options={'gtol': 1e-4, 'maxiter': 25, 'disp': True})
    end_time = time.time()
    print "eclipse time [second] : ", end_time - start_time
    
    optimal_param = res.x
    print scan_name, " : Objective Func : ", res.fun
    print " shape : ", optimal_param[27:]
    
    #------------------------------------------------------------------------------------
    # save
    model.betas[:]    = optimal_param[27:]
    model.pose[48:60] = optimal_param[:12]
    model.pose[3:18]  = optimal_param[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/male02/' + scan_name + '.obj', 'w') as fp:
        for v in model.r: # [m]
            fp.write( 'v %f %f %f\n' % ( v[0], v[1], v[2]) )
        for f in model.f+1: # Faces are 1-based, not 0-based in obj files
            fp.write( 'f %d %d %d\n' %  (f[0], f[1], f[2]) )

    optimal_param = np.zeros(82)
    optimal_param[72:]   = res.x[27:]
    optimal_param[48:60] = res.x[:12]
    optimal_param[3:18]  = res.x[12:27]

    with open(out_folder + 'scan2Optimized_shape_pose/male_param02/' + scan_name + '.pkl', 'w') as fp:
        pickle.dump({'param_data': optimal_param}, fp)


eclipse time [second] :  3020.19609809
male_0112  : Objective Func :  17.598539855064733
 shape :  [ 0.26804486 -0.6037645  -0.02721399 -0.00212397 -0.06065365 -0.07064987
  0.07638807  0.04184382  0.02543404 -0.02840878]
eclipse time [second] :  1758.65068793
male_0015  : Objective Func :  17.330102491540426
 shape :  [ 1.99375894  0.49170971 -0.14680769 -0.29114623  0.02982369 -0.65622253
  0.14997559  0.14036861 -0.05547102 -0.12178027]
eclipse time [second] :  698.482453108
male_0074  : Objective Func :  16.528648928160532
 shape :  [ 2.33717493e-01  6.19349865e-01 -2.39644237e-01 -6.07718118e-04
 -1.14124562e-01 -1.66852076e-01  3.74046312e-02  4.84956659e-02
  2.11616203e-02 -9.92469952e-03]
eclipse time [second] :  678.275201082
male_0110  : Objective Func :  18.819463771890273
 shape :  [ 0.00242352 -0.20715969  0.01509661 -0.1038823   0.06009093 -0.07931821
  0.02808003  0.06854781 -0.01559662 -0.01563914]
eclipse time [second] :  978.307816029
male_0064  : Objective Func :  1

eclipse time [second] :  2309.13990617
male_0061  : Objective Func :  15.40866161274286
 shape :  [ 1.9204358   0.69091485 -0.48123391 -0.22718754 -0.12190176 -0.6071936
 -0.01804883  0.14838044  0.00347306 -0.01920359]
eclipse time [second] :  822.346047878
male_0047  : Objective Func :  19.143320785431975
 shape :  [ 0.67186892 -0.48337444  0.13010358 -0.14135925  0.06344634 -0.15513987
  0.04680848  0.04419986 -0.04500091  0.00731171]
eclipse time [second] :  2557.09568906
male_0037  : Objective Func :  15.256154843706653
 shape :  [ 7.58691830e-01  1.01626694e+00 -1.84874160e-01 -4.32961709e-02
 -1.70876742e-01 -4.49970682e-01  4.27366128e-02  3.80054573e-02
 -8.29160579e-02  9.60883309e-04]
eclipse time [second] :  1380.71772408
male_0114  : Objective Func :  12.549146299866837
 shape :  [-0.7199002  -0.47309891  0.01206152 -0.06698455 -0.06853659 -0.0325127
  0.05048736  0.07605489 -0.00716667  0.02185027]
eclipse time [second] :  818.985414982
male_0079  : Objective Func :  16.7

eclipse time [second] :  1479.82509208
male_0021  : Objective Func :  12.157741175114856
 shape :  [-1.62340962  0.06775818 -0.2120714  -0.40874471 -0.09155316 -0.19557577
  0.37062104  0.19293332  0.02126132 -0.0509927 ]
eclipse time [second] :  1138.733603
male_0063  : Objective Func :  21.109437230428348
 shape :  [ 0.22194947 -0.31636777 -0.00821475 -0.07588773  0.00746184 -0.08005278
  0.03475079  0.03640323 -0.00852521  0.01682733]
eclipse time [second] :  3632.77625394
male_0173  : Objective Func :  15.329928211742411
 shape :  [ 0.60932542  0.51530015 -0.2365867  -0.01903265 -0.08911143 -0.2214127
  0.07605493  0.06528143  0.02357137 -0.03101632]
eclipse time [second] :  620.881778002
male_0035  : Objective Func :  14.558010881159294
 shape :  [-0.21214601  0.61326495 -0.25912178  0.09137139 -0.05685758 -0.07854744
  0.02477428  0.0140891  -0.01842834 -0.00941484]
eclipse time [second] :  2214.04636884
male_0168  : Objective Func :  20.108621247138416
 shape :  [-0.36143444 -0.

eclipse time [second] :  2066.77295589
male_0125  : Objective Func :  19.52225365930619
 shape :  [ 0.29685667 -0.56248567  0.01245268 -0.17682084  0.04376695 -0.11559138
  0.0535336   0.0842464   0.00169167  0.00260232]
eclipse time [second] :  1124.60717201
male_0020  : Objective Func :  18.41154101964858
 shape :  [-0.32195276  0.43944402 -0.06995778  0.00169209  0.02818671 -0.0902992
  0.01737615  0.05990323 -0.0065138   0.00155662]
eclipse time [second] :  1249.07325387
male_0068  : Objective Func :  14.374096960133688
 shape :  [ 0.28377884  0.01209679 -0.05251104 -0.02645588 -0.07513615 -0.04810605
  0.05098087  0.01320784 -0.00206105  0.01506219]
eclipse time [second] :  3506.03096986
male_0128  : Objective Func :  16.811213379692006
 shape :  [ 0.04090497  0.23150997 -0.05917942  0.02138459 -0.01376089 -0.04948555
 -0.00028396  0.00517453 -0.00115939 -0.00263646]
eclipse time [second] :  1062.45820904
male_0019  : Objective Func :  14.801124098653874
 shape :  [ 0.02080059  0.

eclipse time [second] :  1917.14611006
male_0148  : Objective Func :  14.84249121980948
 shape :  [ 0.33906564 -0.30567486 -0.02220275 -0.06047364 -0.04337957 -0.09462409
  0.10408498  0.01857014 -0.01294724  0.02827408]
eclipse time [second] :  2889.54254699
male_0126  : Objective Func :  18.82588861295196
 shape :  [ 0.17272411 -0.64996783 -0.45454999 -0.61155079  0.46747879 -0.51448341
  0.50708107  0.51856013  0.44848204 -0.4555127 ]
eclipse time [second] :  1560.73474884
male_0024  : Objective Func :  20.543524889597123
 shape :  [ 0.2393754  -0.2143491  -0.04964683 -0.05206756  0.01043416 -0.07454185
  0.04656952  0.04940912 -0.00319292 -0.0012109 ]
eclipse time [second] :  3974.86559415
male_0008  : Objective Func :  17.794854942526765
 shape :  [ 1.43282787  1.18168108 -1.59877702  0.78552698 -0.59145098 -1.65853425
  0.99933284  0.52930779 -0.50646605 -0.64571136]
eclipse time [second] :  3148.82154107
male_0153  : Objective Func :  11.37702748771327
 shape :  [ 0.69271     0.

In [5]:
print "done"

done
